In [1]:
import numpy as np # type: ignore
import pandas as pd # type: ignore
from idd_forecast_mbp import constants as rfc
from idd_forecast_mbp.helper_functions import merge_dataframes, read_income_paths, read_urban_paths, level_filter
from idd_forecast_mbp.parquet_functions import read_parquet_with_integer_ids, write_parquet

FORECASTING_DATA_PATH = rfc.MODEL_ROOT / "04-forecasting_data"
PROCESSED_DATA_PATH = rfc.MODEL_ROOT / "02-processed_data"
aa_full_population_df_path = f"{PROCESSED_DATA_PATH}/aa_2023_full_population_df.parquet"

hierarchy_df_path = f'{PROCESSED_DATA_PATH}/full_hierarchy_lsae_1209.parquet'
hierarchy_df = read_parquet_with_integer_ids(hierarchy_df_path)

dah_df_path = f"{PROCESSED_DATA_PATH}/dah_df_2025_07_08.parquet"

In [2]:
new_dah_path = '/mnt/share/resource_tracking/forecasting/dah_channel_HFA/FGH_2024_submission_5_reference/dah_by_channel_hfa_recip_1990_2100.csv'
new_dah_df = pd.read_csv(new_dah_path)
new_dah_df = new_dah_df[(new_dah_df['hfa'] == 'mal') & (new_dah_df['year'] >= 2000)]
new_dah_df = new_dah_df.groupby(['year', 'recip']).agg({'dah': 'sum'}).reset_index()
new_dah_df = new_dah_df.rename(columns={'recip': 'iso3', 'dah': 'mal_DAH_total', 'year': 'year_id'})

A0_hierarchy_df = hierarchy_df[hierarchy_df['level'] == 3].copy()
A0_hierarchy_df = A0_hierarchy_df[['location_id', 'location_name', 'ihme_loc_id']].drop_duplicates().reset_index(drop=True)
A0_hierarchy_df = A0_hierarchy_df.rename(columns={'ihme_loc_id': 'iso3'})

new_dah_df = new_dah_df.merge(A0_hierarchy_df, on='iso3', how='inner')
A0_location_filter = ('location_id', 'in', A0_hierarchy_df['location_id'].unique().tolist())
pop_df = read_parquet_with_integer_ids(aa_full_population_df_path, filters=[A0_location_filter])
new_dah_df = new_dah_df.merge(pop_df, on=['location_id', 'year_id'], how='left')

new_dah_df['mal_DAH_total_per_capita'] = new_dah_df['mal_DAH_total'] / new_dah_df['population']

In [3]:
write_parquet(new_dah_df,dah_df_path)

✅ Metadata validation passed for /mnt/team/idd/pub/forecast-mbp/02-processed_data/dah_df_2025_07_08.parquet


True